In [79]:
import pandas as pd
import ast
import re
from pandas import json_normalize

In [80]:
df_events = pd.read_csv('./Data/liga brasil/events.csv')
df_events.head()

,type,minute,extra,player,club,action,game_id
0,Goals,70,NaN,{'href': '/biel/profil/spieler/875328'},"{'name': 'Esporte Clube Bahia', 'href': '/espo...","{'result': '0:1', 'description': ', Right-foot...",4316187
1,Goals,72,NaN,{'href': '/wesley/profil/spieler/674732'},"{'name': 'Sport Club Internacional', 'href': '...","{'result': '1:1', 'description': ', Left-foote...",4316187
2,Goals,83,NaN,{'href': '/fernando/profil/spieler/51174'},"{'name': 'Sport Club Internacional', 'href': '...","{'result': '2:1', 'description': ', Header, 1....",4316187
3,Substitutions,46,NaN,{'href': '/lucca/profil/spieler/982549'},"{'name': 'Sport Club Internacional', 'href': '...","{'result': None, 'description': ', Tactical', ...",4316187
4,Substitutions,46,NaN,{'href': '/oscar-estupinan/profil/spieler/3882...,"{'name': 'Esporte Clube Bahia', 'href': '/espo...","{'result': None, 'description': ', Tactical', ...",4316187


In [81]:
# Función para convertir texto a diccionario y extraer el número de 'href'
def extraer_id(texto):
    try:
        # Convertir texto a diccionario
        diccionario = ast.literal_eval(texto)
        # Obtener el valor de 'href'
        href = diccionario.get('href', '')
        # Usar una expresión regular para extraer el número
        match = re.search(r'\d+', href)
        return int(match.group(0)) if match else None
    except (ValueError, SyntaxError):
        return None

# Aplicar la función a la columna 'player'
df_events['player_id'] = df_events['player'].apply(extraer_id)
df_events['player_id'] = df_events['player_id'].astype(int)
df_events = df_events.drop('player',axis=1)

df_events.head()

,type,minute,extra,club,action,game_id,player_id
0,Goals,70,NaN,"{'name': 'Esporte Clube Bahia', 'href': '/espo...","{'result': '0:1', 'description': ', Right-foot...",4316187,875328
1,Goals,72,NaN,"{'name': 'Sport Club Internacional', 'href': '...","{'result': '1:1', 'description': ', Left-foote...",4316187,674732
2,Goals,83,NaN,"{'name': 'Sport Club Internacional', 'href': '...","{'result': '2:1', 'description': ', Header, 1....",4316187,51174
3,Substitutions,46,NaN,"{'name': 'Sport Club Internacional', 'href': '...","{'result': None, 'description': ', Tactical', ...",4316187,982549
4,Substitutions,46,NaN,"{'name': 'Esporte Clube Bahia', 'href': '/espo...","{'result': None, 'description': ', Tactical', ...",4316187,388266


In [82]:
# Aplica ast.literal_eval a cada elemento en la columna 'club'
df_events['club'] = df_events['club'].apply(lambda x: ast.literal_eval(x))

# Extrae el valor asociado a la clave 'name' y asígnalo a una nueva columna 'club_name'
df_events['club_name'] = df_events['club'].apply(lambda x: x['name'])

df_events.drop('club',axis=1,inplace=True)

In [83]:
df_events.head()

,type,minute,extra,action,game_id,player_id,club_name
0,Goals,70,NaN,"{'result': '0:1', 'description': ', Right-foot...",4316187,875328,Esporte Clube Bahia
1,Goals,72,NaN,"{'result': '1:1', 'description': ', Left-foote...",4316187,674732,Sport Club Internacional
2,Goals,83,NaN,"{'result': '2:1', 'description': ', Header, 1....",4316187,51174,Sport Club Internacional
3,Substitutions,46,NaN,"{'result': None, 'description': ', Tactical', ...",4316187,982549,Sport Club Internacional
4,Substitutions,46,NaN,"{'result': None, 'description': ', Tactical', ...",4316187,388266,Esporte Clube Bahia


In [85]:
# Aplica ast.literal_eval a cada elemento en la columna 'action'
df_events['action'] = df_events['action'].apply(lambda x: ast.literal_eval(x))

# Usar json_normalize para aplanar los diccionarios
df_normalized = json_normalize(df_events['action'])

# Concatenar el DataFrame original con el DataFrame normalizado
df_events = pd.concat([df_events, df_normalized], axis=1)

# Eliminar la columna original de diccionarios si ya no es necesaria
df_events.drop(columns=['action'], inplace=True)

df_events.head()

,type,minute,extra,game_id,player_id,club_name,result,description,player_in.href,player_assist.href
0,Goals,70,NaN,4316187,875328,Esporte Clube Bahia,0:1,", Right-footed shot, 1. Goal of the Season Ass...",None,/jean-lucas/leistungsdatendetails/spieler/5788...
1,Goals,72,NaN,4316187,674732,Sport Club Internacional,1:1,", Left-footed shot, 1. Goal of the Season",None,None
2,Goals,83,NaN,4316187,51174,Sport Club Internacional,2:1,", Header, 1. Goal of the Season Assist: , Corn...",None,/bruno-henrique/leistungsdatendetails/spieler/...
3,Substitutions,46,NaN,4316187,982549,Sport Club Internacional,None,", Tactical",/wesley/profil/spieler/674732,None
4,Substitutions,46,NaN,4316187,388266,Esporte Clube Bahia,None,", Tactical",/cauly/profil/spieler/162812,None


In [86]:
df_events.to_csv('./Data/liga brasil/events.csv',index=False)